In [4]:
# 读取colmap的bin文件
import numpy as np
import struct
# 设置工作路径
import os
os.chdir('/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/')

import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 读取txt文件

In [6]:
cameras_txt_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/colmap_text/cameras.txt'
images_txt_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/colmap_text/images.txt'
points3D_txt_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/colmap_text/points3D.txt'

from scene.colmap_loader import read_points3D_text, read_intrinsics_text, read_extrinsics_text
xyzs, rgbs, errors = read_points3D_text(points3D_txt_dir)
cameras = read_intrinsics_text(cameras_txt_dir)
images = read_extrinsics_text(images_txt_dir)
# np.unique(images[1].point3D_ids).shape

In [7]:
# 查看xyzs_DBSCAN在xyzs中的id
# xyzs_DBSCAN
# 构建元素到索引的字典
def get_element_to_index(xyzs, rgbs, xyzs_selected) ->tuple[np.ndarray, np.ndarray]:
    element_to_index = {tuple(row): idx for idx, row in enumerate(xyzs)}

    # 生成数组c
    ids = np.array([element_to_index[tuple(row)] for row in xyzs_selected])

    # 输出c的shape
    # print("c.shape =", c.shape)
    # print("c =", c)  # 输出：c = [2 0 1]
    rgbs_selected = np.array([rgbs[i] for i in ids])
    # print("color_after_DBSCAN.shape =", color_after_DBSCAN.shape)
    return ids, rgbs_selected


In [8]:
def read_xyzs_and_rgbs_from_points3D_save_txt(xyzs, rgbs, save_dir = 'txt_files', file_name='xyz_rgb.txt'):
    os.makedirs(save_dir, exist_ok=True)
    with open(os.path.join(save_dir, file_name), 'w') as f:
        for i in range(xyzs.shape[0]):
            f.write('{} {} {} {} {} {}\n'.format(xyzs[i, 0], xyzs[i, 1], xyzs[i, 2], int(rgbs[i, 0]), int(rgbs[i, 1]), int(rgbs[i, 2])))
    print('saved to :', os.path.join(save_dir, file_name))

#### 用此可视化，保存xyzrgb的txt，可用meshlab打开查看

In [9]:
## 读取transforms.json文件，提取相机的世界坐标系与外参(用于基于相机坐标，缩小点的范围)
import json
import numpy as np

json_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/transforms.json'

def get_world_coords_for_cam_centers(json_dir):# 读取json文件
    with open(json_dir, 'r') as f:
        data = json.load(f)

    extrinsics_list = []
    frames_data = data['frames']
    for frame in frames_data:
        R = np.array(frame['R'])
        t = np.array(frame['t'])
        # 转置 t 并调整为列向量（3x1），然后与 r 水平拼接
        top = np.hstack([R, t.reshape(3, 1)])  # 3x4

        # 创建最后一行 [0,0,0,1]
        bottom = np.array([[0, 0, 0, 1]])

        # 垂直拼接生成 4x4 矩阵
        extrinsics = np.vstack([top, bottom])
        extrinsics_list.append(extrinsics)
    return np.asarray(extrinsics_list)


extrinsics_camera = get_world_coords_for_cam_centers(json_dir=json_dir)
R_cameras = extrinsics_camera[:, :3, :3]
T_cameras = extrinsics_camera[:, :3, 3]
camera_center_coords = -(R_cameras.transpose(0,2,1) @ T_cameras[..., np.newaxis]).squeeze(-1)  # 世界坐标系下深度为0的点, 即，中心点
camera_center_coords.shape

(57, 3)

In [91]:
def keep_and_remove_points_org_green(a, b, a_org_rgb=None, save_dir = None, file_name=None):
    '''
    a为所有点，b为相机坐标点，a_org_rgb为原始点云的rgb值，需要基于去除a噪点后，重新排布a的rgb颜色
    '''
    # 计算b的轴对齐包围盒范围
    min_vals = np.min(b, axis=0)
    max_vals = np.max(b, axis=0)
    min_x, min_y, min_z = min_vals
    max_x, max_y, max_z = max_vals
    
    min_xyz = np.min(min_vals)
    max_xyz = np.max(max_vals)
    print('min_xyz:', min_xyz,'max_xyz:', max_xyz)

    # 过滤a的点：保留范围内的点，移除范围外的点
    # mask_a = (
    #     (a[:, 0] >= min_x) & (a[:, 0] <= max_x) &
    #     (a[:, 1] >= min_y) & (a[:, 1] <= max_y) &
    #     (a[:, 2] >= min_z) & (a[:, 2] <= max_z)
    # )
    mask_a = (
        (a[:, 0] >= min_xyz) & (a[:, 0] <= max_xyz) &
        (a[:, 1] >= min_xyz) & (a[:, 1] <= max_xyz) &
        (a[:, 2] >= min_xyz) & (a[:, 2] <= max_xyz)
    )
    a_inside = a[mask_a]    # a中的保留点
    a_outside = a[~mask_a]  # a中的移除点

    # b的所有点均会被保留（因为包围盒由b生成）
    b_inside = b  # b中所有点均在包围盒内

    ids, a_rgbs_selected = get_element_to_index(a, a_org_rgb, a_inside)
    # 合并保留的点（红色）和移除的点（蓝色）
    combined_inside = np.vstack([a_inside, b_inside])  # 保留的点
    # print('combined_inside.shape:', combined_inside.shape)

    # print('a_org_rgb.shape:', a_org_rgb.shape)
    # red_colors = np.tile([1, 0, 0], (combined_inside.shape[0], 1))  # 红色：保留点
    org_colors = (a_org_rgb[ids]) / 255.0  # 原始点云的rgb值
    # print('org_colors.shape:', org_colors.shape)
    
    magenta_colors = np.tile([1, 0, 1], (b_inside.shape[0], 1))
    combined_inside_colors = np.vstack([org_colors, magenta_colors])  # 合并的点的rgb值
    blue_colors = np.tile([0, 0.5, 0], (a_outside.shape[0], 1))        # 绿色：移除点

    # 创建点云对象并设置颜色
    pcd_inside = o3d.geometry.PointCloud()
    pcd_inside.points = o3d.utility.Vector3dVector(combined_inside)
    pcd_inside.colors = o3d.utility.Vector3dVector(combined_inside_colors)

    pcd_outside = o3d.geometry.PointCloud()
    pcd_outside.points = o3d.utility.Vector3dVector(a_outside)
    pcd_outside.colors = o3d.utility.Vector3dVector(blue_colors)

    point_all = np.vstack([combined_inside, a_outside])
    color_all = np.vstack([combined_inside_colors, blue_colors])
    
    point_selected = a_inside
    color_selected = a_rgbs_selected
    # 创建包围盒线框模型
    # vertices = np.array([
    #     [min_x, min_y, min_z], [max_x, min_y, min_z],
    #     [max_x, max_y, min_z], [min_x, max_y, min_z],
    #     [min_x, min_y, max_z], [max_x, min_y, max_z],
    #     [max_x, max_y, max_z], [min_x, max_y, max_z]
    # ])
    vertices = np.array([
        [min_xyz, min_xyz, min_xyz], [max_xyz, min_xyz, min_xyz],
        [max_xyz, max_xyz, min_xyz], [min_xyz, max_xyz, min_xyz],
        [min_xyz, min_xyz, max_xyz], [max_xyz, min_xyz, max_xyz],
        [max_xyz, max_xyz, max_xyz], [min_xyz, max_xyz, max_xyz]
    ])
    lines = [[0,1], [1,2], [2,3], [3,0], [4,5], [5,6], [6,7], [7,4], [0,4], [1,5], [2,6], [3,7]]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(vertices),
        lines=o3d.utility.Vector2iVector(lines)
    )
    line_set.paint_uniform_color([1, 0, 0])  # 红色包围盒边框
    print('box内部的点个数：', a_inside.shape[0], org_colors.shape[0], org_colors[0], org_colors[1])
    # 可视化展示
    # 设置黑色背景
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='3D Viewer', width=1280, height=720, left=0, top=0)
    # 设置点的大小
    vis.get_render_option().point_size = 2.0
    vis.add_geometry(pcd_inside)
    vis.add_geometry(pcd_outside)
    vis.add_geometry(line_set)
    
    # 设置颜色

    vis.get_render_option().background_color = np.array([0, 0, 0])  # 设置黑色背景
    vis.run()
    vis.destroy_window()

    # 展示  
    # o3d.visualization.draw_geometries([pcd_inside, pcd_outside, line_set])
    
    # 是否存储
    if save_dir and file_name is not None:
        # 展示用
        read_xyzs_and_rgbs_from_points3D_save_txt(combined_inside, combined_inside_colors, save_dir=save_dir, file_name=file_name)
        # open3d存储[pcd_inside, pcd_outside, line_set]
        read_xyzs_and_rgbs_from_points3D_save_txt(point_all, color_all, save_dir=save_dir, file_name=file_name.split('.')[0]+'_all.txt')
        # o3d.io.write_point_cloud(os.path.join(save_dir, file_name.split('.')[0]+'.ply'), pcd_all)
        read_xyzs_and_rgbs_from_points3D_save_txt(point_selected, color_selected, save_dir=save_dir, file_name=file_name.split('.')[0]+'_selected.txt')
    else:
        print('Not save the txt file.')
    list_xyz_min_max = [min_x, max_x, min_y, max_y, min_z, max_z]
    return point_selected, color_selected, min_xyz, max_xyz, vertices, list_xyz_min_max, ids

In [ ]:
save_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/colmap_text'
file_name = 'camera_box_selected_xyz_rgb.txt'
outputs_tuple = keep_and_remove_points_org_green(a=xyzs, b = camera_center_coords, a_org_rgb=rgbs, \
    save_dir=save_dir, file_name=file_name)

In [60]:
ids_boxed = outputs_tuple[-1]
print('boxed len(ids_boxed):', len(ids_boxed), ids_boxed, sep='\n')

# 在colmap生成的point3D.txt文件中根据ids选取对应的行
def read_points3d_txt_select_as_ids_to_new_points3d_txt(points3D_txt_dir, ids):
    with open(points3D_txt_dir, 'r') as f:
        f_list = f.readlines()
        f_new_list = []
        # f_new_list_compared = []
        for id in ids.tolist():
            for f_line in f_list:
                if f_line.split()[0] == str(id):
                    f_new_list.append(f_line)
            # f_new_list.append(f_list[id+3])
            # f_new_list_compared.append(f'{id} {f_list[id+2]}')
    return f_new_list#, f_new_list_compared

boxed len(ids_boxed):
20873
[    0     1     2 ... 32809 32811 32812]


In [55]:
points3D_txt_dir = '/home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/colmap_text/points3D.txt'
save_dir = os.path.join(os.path.dirname(points3D_txt_dir), 'points3D_select.txt')

# boxed_list = read_points3d_txt_select_as_ids_to_new_points3d_txt(points3D_txt_dir=points3D_txt_dir, ids=ids_boxed)

# with open(save_dir, 'w') as f_new:
#     f_new.write(''.join(boxed_list))
# print('Camera poses boxed and saved to:', save_dir)

Camera poses boxed and saved to: /home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/colmap_text/points3D_select.txt


In [70]:
with open(points3D_txt_dir, "r") as fid:
    lines = fid.readlines()

In [75]:
lines[3].split()[1]

'2.6844834692200816'

In [82]:
def use_boxed_xyzs_to_select_from_points3D_txt(points3D_txt_list:list, xyzs:np.array)->list:

    boxed_xyzs = []
    for x in xyzs:
        x_str = str(x[0].tolist())
        for line in points3D_txt_list:
            if x_str[:-3] in line.split()[1]:
                boxed_xyzs.append(line)
                # break
            else:
                continue
    return boxed_xyzs



In [83]:
boxed_xyzs = use_boxed_xyzs_to_select_from_points3D_txt(points3D_txt_list=lines, xyzs=xyz_point_boxed )

In [87]:
print(len(boxed_xyzs), len(set(boxed_xyzs)))
boxed_xyzs_list = list(set(boxed_xyzs))

22908 20873


In [88]:
with open(save_dir, 'w') as f_new:
    f_new.write(''.join(boxed_xyzs_list))
print('Camera poses boxed and saved to:', save_dir)

Camera poses boxed and saved to: /home/ubunto/Project/konglx/pcd/2dgs/2d-gaussian-splatting-main/datasets/dalian_xinghaiwandaqiao_video_input_rgb__camera_box/colmap_text/points3D_select.txt
